# Imports

In [1]:
import os
import pathlib
import random
from math import sqrt
import ujson

import pandas as pd
import plotly.express as px

PATH = pathlib.Path.cwd() / "src"

# set current working directory to the path
os.chdir(PATH)
print(f"Current path: {PATH}")

Current path: /Users/eliebrosset/Documents/GitHub/cymoo/src


In [2]:
from MOO import MOO
from optimizers.NSGA2 import NSGA2
from optimizers.NSRA import NSRA
from optimizers.NSWGE import NSWGE
from problems.multitask_routing.Network import Network
from problems.multitask_routing.Task import Task
from problems.Solution import Solution

# Defining the parameters

## Layers

In [3]:
# PROBLEM DEFINITION
paramsLayerOne = {
    "unit": {
        "tag": "DEVICE",
        "computingSpeed": lambda: random.randint(5, 10),
        "positionX": 0,
        "positionY": 0,
        "throughput": 0.5,
        "pollution": lambda: round(random.random(), 2),
        "cost": lambda: round(random.random(), 2),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 1,
        "flowRate": lambda: 1,
    },
    "numberNewUnits": 10,
}

paramsLayerTwo = {
    "unit": {
        "tag": "FOG",
        "computingSpeed": lambda x: x.computingSpeed * random.randint(5, 10),
        "positionX": lambda x: x.positionX + random.randint(-2, 2),
        "positionY": lambda x: x.positionY + random.randint(-2, 2),
        "throughput": lambda x: x.throughput * round(random.random(), 2) * 2
        + 1,
        "pollution": lambda x: (x.pollution + 1)
        * (round(random.random(), 2) + 1),
        "cost": lambda x: (x.cost + 1) * (round(random.random(), 2) + 1),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 2,
        "flowRate": lambda: 3,
    },
    "numberNewUnits": 20,
}

paramsLayerThree = {
    "unit": {
        "tag": "CLOUD",
        "computingSpeed": lambda x: x.computingSpeed * random.randint(50, 100),
        "positionX": lambda x: x.positionX + random.randint(-5, 5),
        "positionY": lambda x: x.positionY + random.randint(-5, 5),
        "throughput": lambda x: x.throughput * round(random.random(), 2) * 3
        + 1,
        "pollution": lambda x: (x.pollution + 1)
        * (round(random.random(), 2) + 1),
        "cost": lambda x: (x.cost + 1) * (round(random.random(), 2) + 1),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 2,
        "flowRate": lambda: 3,
    },
    "numberNewUnits": 10,
}

## Tasks

In [4]:
tasks = tuple(
    Task(random.randint(1, 10000), random.randint(1, 10000)) for _ in range(3)
)

## Network

In [5]:
optimUnits = {
    "processingTime": "Processing time (s)",
    "cost": "Cost (€)",
    "pollution": "Pollution (g)",
}
optimDirections = {
    "processingTime": "min",
    "cost": "min",
    "pollution": "min",
}
problem = Network(
    "DEVICE",
    tasks=tasks,
    optimDirections=optimDirections,
    minDepth=1,
    maxDepth=15,
    mutationRate=0.1,
    layers=[paramsLayerOne, paramsLayerTwo, paramsLayerThree],
)

## Optimization

In [6]:
nIterations = 10
nSolutions = 1000
seed = 10

# Optimization

## Training

In [7]:
moo = MOO(problem)

nsga2_paretos = moo.optimize(
    NSGA2,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
    seed=seed,
    ratioKept=0.5,
)
nsra_paretos = moo.optimize(
    NSRA,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
    seed=seed,
    ratioKept=0.5,
)
# nswge_paretos = moo.optimize(
#     NSWGE,
#     nSolutions,
#     nIterations,
#     imDir=PATH.parent / "imgs",
#     saveDir=PATH.parent / "paretos",
#     export=True,
#     seed=seed,
# )

Optimizing with NSGA2...


100%|██████████| 10/10 [00:06<00:00,  1.63it/s]


Displaying pareto solutions...
{'processingTime': 17959.0, 'cost': 10627.147913627125, 'pollution': 2056.513794202822}
{'processingTime': 18044.285714285714, 'cost': 7158.68200287961, 'pollution': 15397.203771042427}
{'processingTime': 17959.0, 'cost': 22692.259601843147, 'pollution': 10484.48816826414}
{'processingTime': 18044.285714285714, 'cost': 6330.375697594665, 'pollution': 19277.846447090087}
{'processingTime': 17959.0, 'cost': 13171.412422616006, 'pollution': 4256.726536671413}
{'processingTime': 17959.0, 'cost': 9970.061916167664, 'pollution': 4139.473205256154}
{'processingTime': 17959.0, 'cost': 8506.201945179382, 'pollution': 3918.2237290138087}
{'processingTime': 17959.0, 'cost': 12831.31139326451, 'pollution': 10455.722809018676}

Optimizing with NSRA...


100%|██████████| 10/10 [00:06<00:00,  1.48it/s]


Displaying pareto solutions...
{'processingTime': 46236.3296163646, 'cost': 32445.918641837092, 'pollution': 28883.906065658233}
{'processingTime': 47764.676309637594, 'cost': 19762.36966863167, 'pollution': 33788.84891866593}
{'processingTime': 45935.89885937625, 'cost': 28178.830932107954, 'pollution': 29622.249129467808}
{'processingTime': 46006.014979542124, 'cost': 62205.582731164686, 'pollution': 28971.95118243332}
{'processingTime': 47397.77178226322, 'cost': 62397.64559043612, 'pollution': 28276.883212729623}
{'processingTime': 71239.0, 'cost': 70343.3529831825, 'pollution': 27652.78904788046}



## Evaluation

In [8]:
MOO.relative_efficiency(nsra_paretos, nsga2_paretos, Solution.optimDirections, verbose=True)

Relative efficiency: 0 % of <class 'optimizers.NSRA.NSRA'> solutions are undominated by <class 'optimizers.NSGA2.NSGA2'> solutions
Relative efficiency: 100 % of <class 'optimizers.NSGA2.NSGA2'> solutions are undominated by <class 'optimizers.NSRA.NSRA'> solutions


(0, 1)

# Results

In [9]:
def generate_violin_plot(df, title, yaxis_title="Value"):
    fig = px.violin(
        df,
        x="Epoch",
        y="Value",
        box=True,
        points=False,
        hover_data=df.columns,
        template="plotly_white",
        color_discrete_sequence=["#1f77b4"],
    )
    fig.update_layout(
        title=title,
        xaxis_title="Epoch",
        yaxis_title=yaxis_title,
        font=dict(
            family="Times New Roman",
            size=12,
        ),
    )
    fig.show()

In [10]:
# load pareto data from json
for file in os.listdir(PATH.parent / "paretos"):
    if file.endswith(".json"):
        # load file as dict
        print(file)
        with open(PATH.parent / "paretos" / file, "r") as f:
            data = ujson.load(f)
        for objective in optimDirections.keys():
            # unstack values
            df = (
                pd.DataFrame.from_dict(data[objective], orient="index")
                .stack()
                .reset_index()
                .rename(columns={"level_0": "Epoch", 0: "Value"})
                .drop(columns="level_1")
            )
            # generate plot
            generate_violin_plot(
                df,
                title=f"Violin plot of the pareto solutions for each epoch (10 epochs) for the {objective} objective",
                yaxis_title=optimUnits[objective],
            )


NSGA2.json


NSRA.json
